In [ ]:
from clickhouse_driver import Client

In [ ]:
client = Client(
    host='localhost',
    port=9000,
    database='fleet',
    user='fleet',
    password='fleet',
    settings={'use_numpy': True},
)

In [ ]:
longitude, latitude = client.execute(
    "select longitude, latitude "
    "from get_flight_endpoints(start_time=now() - interval '1 day', end_time=now()) "
    "where on_ground = true ",
    columnar=True,
)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
landings = pd.DataFrame({'longitude': longitude, 'latitude': latitude})
landings = landings.astype({'longitude': np.float64, 'latitude': np.float64})

In [ ]:
landings.head()

In [ ]:
len(landings)

In [ ]:
import folium

In [ ]:
map = folium.Map([50, 10], zoom_start=5, control_scale=True)
landings.sample(100).apply(
    lambda row: folium.Marker([row['latitude'], row['longitude']]).add_to(map),
    axis=1,
)
map

In [ ]:
import cartopy as cp
from sklearn.cluster import DBSCAN

In [ ]:
projection = cp.crs.Robinson()
geodetic = cp.crs.Geodetic()
projected = projection.transform_points(geodetic, landings['longitude'], landings['latitude'])
landings['x'], landings['y'] = projected.T[:2]

In [ ]:
projection.x_limits, projection.y_limits

In [ ]:
dbscan = DBSCAN(eps=5_000, min_samples=5).fit(landings[['x', 'y']])
landings['airport'] = dbscan.labels_

In [ ]:
landings = landings[landings['airport'] >= 0]

In [ ]:
landings.head()

In [ ]:
len(landings)

In [ ]:
airports = landings.groupby('airport').agg(
    x=('x', 'mean'),
    y=('y', 'mean'),
    landings=('airport', 'size')
)
wgs84 = geodetic.transform_points(projection, airports['x'], airports['y'])
airports['longitude'], airports['latitude'] = wgs84.T[:2]
airports.index.rename('id')

In [ ]:
airports.head()

In [ ]:
len(airports)

In [ ]:
map = folium.Map([50, 10], zoom_start=5, control_scale=True)
airports.apply(
    lambda row: folium.Marker([row['latitude'], row['longitude']]).add_to(map),
    axis=1,
)
map

In [ ]:
longitude, latitude, altitude = client.execute(
    "select longitude, latitude, baro_altitude "
    "from get_altitude_minima(start_time=now() - interval '1 day', end_time=now()) ",
    columnar=True,
)

In [ ]:
minima = pd.DataFrame({'longitude': longitude, 'latitude': latitude, 'altitude': altitude})
minima = minima.astype({'longitude': np.float64, 'latitude': np.float64, 'altitude': np.float32})

In [ ]:
minima.head()

In [ ]:
len(minima)

In [ ]:
import xarray as xr
import pygmt

In [ ]:
elevation = pygmt.datasets.load_earth_relief(resolution='06m')

In [ ]:
elevation.shape, elevation.dtype

In [ ]:
ground_level = elevation.interp(
    lat = xr.DataArray(minima['latitude']),
    lon = xr.DataArray(minima['longitude']),
)
minima['agl'] = minima['altitude'] - ground_level

In [ ]:
minima = minima[minima['agl'] < 250]

In [ ]:
minima.head()

In [ ]:
len(minima)

In [ ]:
from cartopy.geodesic import Geodesic

In [ ]:
geodesic = Geodesic()
endpoints = airports[['longitude', 'latitude']].to_numpy()
dists = minima[['longitude', 'latitude']].apply(
    lambda row: geodesic.inverse(row.to_numpy(), endpoints)[:, 0],
    axis=1,
)
dists = np.stack(dists.to_numpy()).astype(np.float32)

In [ ]:
airport_idx = dists.argmin(axis=1)
minima['airport'] = airports.index.to_numpy()[airport_idx]
minima['airport_dist'] = dists[range(dists.shape[0]), airport_idx]

In [ ]:
minima = minima[minima['airport_dist'] < 10_000]

In [ ]:
minima.head()

In [ ]:
len(minima)

In [ ]:
go_arounds = minima.groupby('airport').size()
airports['go_arounds'] = 0
airports.loc[go_arounds.index, 'go_arounds'] = go_arounds
airports['risk'] = airports['go_arounds'] / airports['landings']

In [ ]:
airports.head()

In [ ]:
dangerous_airports = airports[airports['risk'] > 0]
dangerous_airports = dangerous_airports.sort_values('risk', ascending=False)
dangerous_airports.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_theme()
sns.histplot(dangerous_airports['landings'], log_scale=True)
plt.show()

In [ ]:
dangerous_airports = dangerous_airports[dangerous_airports['landings'] > 100]
dangerous_airports.head()

In [ ]:
def get_color(landings, risk):
    if landings < 100:
        return 'blue'
    if risk < 1/3:
        return 'green'
    if risk < 2/3:
        return 'orange'
    return 'red'
map = folium.Map([50, 10], zoom_start=5, control_scale=True)
add_marker = lambda row: folium.Marker(
    [row['latitude'], row['longitude']],
    popup=f"Risk: {row['risk']:.3f}",
    icon=folium.Icon(color=get_color(row['landings'], row['risk']))
).add_to(map)
airports.apply(
    add_marker,
    axis=1,
)
map

In [ ]:
plt.style.use('default')
plt.figure(figsize=(50, 20))

ax = plt.axes(projection=projection)
# ax.stock_img()
ax.add_feature(cp.feature.OCEAN)
ax.add_feature(cp.feature.LAND)
ax.add_feature(cp.feature.RIVERS)
ax.add_feature(cp.feature.LAKES)
ax.add_feature(cp.feature.COASTLINE)
ax.add_feature(cp.feature.BORDERS, linewidth=0.5)
ax.add_feature(cp.feature.STATES, linewidth=0.5, linestyle='dashed')

scatter = ax.scatter(
    x=airports['longitude'],
    y=airports['latitude'],
    s=(airports['landings'] / 5),
    c=np.minimum(airports['risk'], 1),
    transform=geodetic,
    cmap='RdYlGn_r',
    edgecolor='black',
    linewidth=0.5,
    zorder=2,
)
ax.set_xlim(projection.x_limits)
ax.set_ylim(projection.y_limits)
cbar = plt.colorbar(scatter)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('Risk', fontsize=20)

plt.title('Airport Safety', fontsize=40)
plt.show()